# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

In [79]:
from random import choices, random, randint
from math import ceil
from copy import deepcopy
import lab9_lib

### Genetic Algorithm

Initial settings:
- POP_SIZE = 100
- OFFSPRING_SIZE = 20 🧬
- GENERATIONS = 1000 
- TOURNAMENT_SIZE = 30
- MUTATION_PROBABILITY = 0.5 🎰

There are also 3 other parameters: 
- ADAPTABILITY : is an int that represents the number of generations that the algorithm will run, before the update of some parameters
- TOURNAMENT_MODIFIER = 2, is the value for the self-adaptation of the tournament size
- MUTATION_MODIFIER = 0.03, is the value for the self-adaptation of the mutation probability 


I decided to use a 'plus' strategy, so the offspring will be added to the population. 
The selection is made with a tournament selection approach and the size of the tournament is self-adapted based on the performance of the algorithm. 
The mutation probability is also self-adapted based on the performance of the algorithm.

The main idea of my algorithm is to evaluate the fitness on subchunks of the genome, saving the fitness value of each subchunk (avoiding to recompute it if it is already computed due the fact the fitness function is costly), saving the chuck structure in SUB_CHUNKS_SAVED and the fitness value in SUB_FITNESS_SAVED.

The idea is to maximize the fitness of each subchunk, the length of the subchunk 2*instance. 

The crossover is made by taking the best subchunk between the two parents, this is done for each subchunk of the genome.

In [80]:
SUB_CHUNKS_SAVED = [] ##for optimization reasons 
SUB_FITNESS_SAVED = [] ##for optimization reasons
class Individual:
    def __init__(self, genome,fit, k):
        self.genome = genome
        self.fitness = fit
        if k < 10 :
            k = k * 2
        self.sub_chunks = [self.genome[i:i+k] for i in range(0, len(self.genome), k)]
        self.fitness_chunk = [self.chunk_fitness_alredy_computed(i) for i in range(len(self.sub_chunks))]
    
    def chunk_fitness_alredy_computed(self, chunk_index : int) -> float:
        '''
        Search in the rest of the chunks if the fitness is already computed
        '''
        tmp_chunk = self.sub_chunks[chunk_index]
        if tmp_chunk in SUB_CHUNKS_SAVED:
            return SUB_FITNESS_SAVED[SUB_CHUNKS_SAVED.index(tmp_chunk)]
        else:
            SUB_CHUNKS_SAVED.append(tmp_chunk)
            fit = fitness(tmp_chunk)
            SUB_FITNESS_SAVED.append(fit)
            return fit 

## Settings ⚙️

Because the goal is to reach the best fitness in the minimum number of fitness calls, the starting point is a 'small' population and a 'small' number of offspring, so the algorithm will run for a lot of generations, but the number of fitness calls will be low.

The tournament size and the mutation probability are self-adapted, initially are set to favor the exploration, but with the self-adaptation they can change during the execution of the algorithm and favor the exploitation.
The MUTATION_MODIFIER is the value that is added/subtracted to the mutation probability, the TOURNAMENT_MODIFIER is the value that is added/subtracted to the tournament size.
The ADAPTABILITY is the number of generations that the algorithm will run before the update of the parameters.
The INCREASING_THRESHOLD is the value that the fitness must be increased to favor the exploitation, else the exploration is favored.

In [81]:
instances = [1,2,5,10] 
POP_SIZE = 100
OFFSPRING_SIZE =  20
LOCI = 10000
MUTATION_RATE = 0.7
GENERATIONS = 1000
TOURNAMENT_SIZE = 30 
INCREASING_THRESHOLD = 0.01
MUTATION_MODIFIER = 0.03
TOURNAMENT_MODIFIER = 2
ADAMPTABILITY = 10
MUTATION_DIVIDER = 100
K = instances[2]
fitness = lab9_lib.make_problem(K)

In [82]:
def create_population(pop_size : int, loci : int) -> list:
    population = []
    for _ in range(pop_size):
        genome = choices([0, 1], k=loci)
        fit = fitness(genome)
        population.append(Individual(genome,fit, K))
    return population


### Functions to manage self-adaptation

In the algorithm at each iteration is saved the best fitness value, the adaptive strategy is based on the performance of the algorithm, so if the best fitness value is not improving for a certain number of generations, the algorithm will change the parameters to favor the exploration.

There is also the possibility to be in a flat region, so the algorithm will change the parameters to favor the exploration (big increase of the tournament size and mutation probability).

In [83]:
def adaptive_tournament_size(adaptation : list[float], tournament_size : int) -> int : 
    flat_land = set(adaptation)
    if len(flat_land) == 1:
        tournament_size-=TOURNAMENT_MODIFIER*3    
    elif adaptation[-1] < adaptation[0] + INCREASING_THRESHOLD:
        ## favours exploration
        tournament_size -= TOURNAMENT_MODIFIER
    
    elif adaptation[-1] >= adaptation[0] + INCREASING_THRESHOLD:
        ## favours exploitation
        tournament_size += TOURNAMENT_MODIFIER
    if tournament_size > 80:
        tournament_size =80
    elif tournament_size < 10: 
        tournament_size = 10
    return tournament_size

def adaptive_mutation_rate(adaptation : list[float], mutation_rate : float) -> float : 

    flat_land = set(adaptation)
    if len(flat_land) == 1:
        mutation_rate*=1.3    
        print("Flat land")
    elif adaptation[-1] < adaptation[0] + INCREASING_THRESHOLD:
        ## favours exploration
        mutation_rate += MUTATION_MODIFIER
    
    elif adaptation[-1] >= adaptation[0] + INCREASING_THRESHOLD:
        ## favours exploitation
        mutation_rate -= MUTATION_MODIFIER
    if mutation_rate > 1.0:
        mutation_rate = 1.0
    elif mutation_rate < 0.0:
        mutation_rate = 0.0
    return mutation_rate
    

### Genetic Algorithm

In [84]:
def genetic_algorithm(population : list[Individual], offspring_size : int, tournament_size : int, mutation_rate : float, generations : int) -> Individual:
    list_of_best = []
    current_best = max(population, key=lambda ind: ind.fitness).fitness
    list_of_best.append(current_best) 
    for _ in range(generations):
        offspring = []
        for i in range(offspring_size):
            parent1 = tournament_selection(population, tournament_size)
            parent2 = tournament_selection(population, tournament_size)
            offspring.append(uniform_sub_chuncks_crossover(parent1, parent2))
            
        for index, ind in enumerate(offspring):
            if random() < mutation_rate:
                offspring[index] = mutate_sub_chuncks(ind, mutation_rate)
        pop_size = len(population)
        population += offspring
        population = sorted(population, key = lambda x : x.fitness , reverse=True)[:pop_size]

        if population[0].fitness > current_best + 0.05:
            print(f"Good improvement, fitness increased of {(population[0].fitness - current_best):.2%} ↗️")
        current_best = population[0].fitness
        list_of_best.append(current_best)
        if _ % ADAMPTABILITY == 0 and _ != 0:
            mutation_rate = adaptive_mutation_rate(list_of_best, mutation_rate)
            tournament_size = adaptive_tournament_size(list_of_best, tournament_size)
            list_of_best = [current_best]
        print(f"gen {_} Best fitness: {population[0].fitness:.2%} with mutation rate {mutation_rate:.2%} and tournament size {tournament_size} offspring size {offspring_size}")
        if current_best == 1.0:
            return population[0]
    return max(population, key=lambda ind: ind.fitness)


def tournament_selection(population : list[Individual], tournament_size : int) -> list:
    tournament = choices(population, k=tournament_size)
    return max(tournament, key=lambda ind: ind.fitness)

def uniform_sub_chuncks_crossover(parent1 : Individual, parent2 : Individual) -> Individual:
    genome = []
    for i in range(len(parent1.sub_chunks)):
        if parent1.fitness_chunk[i] >= parent2.fitness_chunk[i]:
            genome += parent1.sub_chunks[i]
        else:
            genome += parent2.sub_chunks[i]
    return Individual(genome=genome,fit=fitness(genome),k=K)

def mutate_sub_chuncks(ind : Individual, mutation_rate : float):
    ind = deepcopy(ind)
    for i in range(len(ind.sub_chunks)):
        if ind.fitness_chunk[i] < 1.0:
            indice = randint(0, len(ind.sub_chunks[i]) - 1) 
            ind.sub_chunks[i][indice] = 1-ind.sub_chunks[i][indice]  ###flip one bit tryin to improve the fitness   
            ind.fitness_chunk[i] = ind.chunk_fitness_alredy_computed(i)
    ind.fitness = sum(ind.fitness_chunk) / len(ind.fitness_chunk)
    return ind


In [85]:
top = genetic_algorithm(create_population(POP_SIZE, LOCI), OFFSPRING_SIZE, TOURNAMENT_SIZE, MUTATION_RATE, GENERATIONS)
print(f"Number of calls: {fitness.calls}")

Good improvement, fitness increased of 16.45% ↗️
gen 0 Best fitness: 35.54% with mutation rate 60.00% and tournament size 30 offspring size 20
gen 1 Best fitness: 37.73% with mutation rate 60.00% and tournament size 30 offspring size 20
gen 2 Best fitness: 40.25% with mutation rate 60.00% and tournament size 30 offspring size 20
gen 3 Best fitness: 42.90% with mutation rate 60.00% and tournament size 30 offspring size 20
gen 4 Best fitness: 45.55% with mutation rate 60.00% and tournament size 30 offspring size 20
gen 5 Best fitness: 46.89% with mutation rate 60.00% and tournament size 30 offspring size 20
gen 6 Best fitness: 49.42% with mutation rate 60.00% and tournament size 30 offspring size 20
gen 7 Best fitness: 50.93% with mutation rate 60.00% and tournament size 30 offspring size 20
gen 8 Best fitness: 53.97% with mutation rate 60.00% and tournament size 30 offspring size 20
gen 9 Best fitness: 55.71% with mutation rate 60.00% and tournament size 30 offspring size 20
gen 10 Best